## What
When you create a simplified interface that performs many other actions behind the scenes. 
Sometimes you may end up with a very large/confusing collection of classes and interactions. It's important to hide 
this complexity from the client via a Facade.

In [7]:
# motivation: build a multi-server OS http://www.helenos.org/doc/slides/2010-02-03-Jermar-Multiserver.pdf 
# source: Mastering Python Design Patterns
from enum import Enum
from abc import ABCMeta, abstractmethod

State = Enum('State', 'new running sleeping restart zombie')

class User:
    pass

class Process:
    pass

class File:
    pass

class AuthenticationServer:
    pass

class NetworkServer:
    pass

class Server(metaclass=ABCMeta):
    # a decorator: way of altering functions/classes w/out having to inherit or subclass
    # forces functions to be named / implemented in subclasses
    @abstractmethod
    def __init__(self):
        pass

    def __str__(self):
        return self.name

    @abstractmethod
    def boot(self):
        pass

    @abstractmethod 
    def kill(self, restart=True):
        pass

class FileServer(Server):
    def __init__(self):
        self.name = 'FileServer'
        self.state = State.new

    def boot(self):
        print('Booting the {}'.format(self))
        self.state = State.running

    def kill(self, restart=True):
        print('Killing {}'.format(self))
        self.state = State.restart if restart else State.zombie

    def create_file(self, user, name, permissions):
        print("Creating file '{}' for user '{}' with permissions {}".format(name, user, permissions))

class ProcessServer(Server):
    def __init__(self):
        self.name = 'ProcessServer'
        self.state = State.new

    def boot(self):
        print('Booting the {}'.format(self))
        self.state = State.running

    def kill(self, restart=True):
        print('Killing {}'.format(self))
        self.state = State.restart if restart else State.zombie

    def create_process(self, user, name):
        print("Creating the process '{}' for user '{}'".format(name, user))

class OperatingSystem:
    '''The Facade'''
    def __init__(self):
        self.fs = FileServer()
        self.ps = ProcessServer()

    def start(self):
        [i.boot() for i in (self.fs, self.ps)]

    def create_file(self, user, name, permissions):
        return self.fs.create_file(user, name, permissions)

    def create_process(self, user, name):
        return self.ps.create_process(user, name)

def main():
    os = OperatingSystem()
    os.start() 
    os.create_file('foo', 'hello', '-rw-r-r')
    os.create_process('bar', 'ls /tmp')

if __name__ == '__main__':
    main()

Booting the FileServer
Booting the ProcessServer
Creating file 'hello' for user 'foo' with permissions -rw-r-r
Creating the process 'ls /tmp' for user 'bar'


## Exercise: Implement one of the methods